Importing libraries and data

In [2]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MultiLabelBinarizer

In [3]:
clean_movies = pd.read_csv('clean_movie_data.csv')
movies = pd.read_csv('movies.csv')

Function to create a clean title

In [5]:
import re
import string

def clean_title(title):
    'This returns a string of a movie title with some regex to better parse names and deal with inconsistencies'
    title = re.sub(r'\s*\(\d{4}\)$', '', title).strip().lower()
    
    match = re.match(r'(.+),\s*(the|a|an)$', title)
    if match:
        title = f"{match.group(2)} {match.group(1)}"  # Move article to the front

    title = title.translate(str.maketrans('', '', string.punctuation))

    return title

Merging the data to include the id associated with user reviews

In [7]:
movies['clean_title'] = movies['title'].apply(clean_title)
clean_movies['clean_title'] = clean_movies['original_title'].apply(clean_title)

movies = movies.drop_duplicates(subset=['clean_title'], keep='first')   #Dropping duplicate titles
clean_movies = clean_movies.merge(movies[['clean_title', 'movieId']], on = 'clean_title', how = 'left') #Joining the data on the clean title
clean_movies = clean_movies.dropna(subset=['movieId'])  # Dropping rows that do not have a valid movie ID (Couldn't find a matching clean title)
clean_movies['movieId'] = clean_movies['movieId'].fillna(0).astype(int) #converting to int


clean_movies = clean_movies.drop(columns=['clean_title'])
clean_movies = clean_movies.set_index('movieId')

In [8]:
clean_movies.head()

,budget,genres,keywords,original_title,popularity,production_companies,release_date,revenue,runtime,vote_average,vote_count,production_ISO,spoken_ISO,release_year,release_month,release_day,revenue_imputed
movieId,,,,,,,,,,,,,,,,,
72998,237000000,"Action, Adventure, Fantasy, Science Fiction","culture clash, future, space war, space colony...",Avatar,150.437577,"Ingenious Film Partners, Twentieth Century Fox...",2009-12-10,2.787965e+09,162.0,7.2,11800,"US, GB","en, es",2009.0,12.0,10.0,2.787965e+09
53125,300000000,"Adventure, Fantasy, Action","ocean, drug abuse, exotic island, east india t...",Pirates of the Caribbean: At World's End,139.082615,"Walt Disney Pictures, Jerry Bruckheimer Films,...",2007-05-19,9.610000e+08,169.0,6.9,4500,US,en,2007.0,5.0,19.0,9.610000e+08
136020,245000000,"Action, Adventure, Crime","spy, based on novel, secret agent, sequel, mi6...",Spectre,107.376788,"Columbia Pictures, Danjaq, B24",2015-10-26,8.806746e+08,148.0,6.3,4466,"GB, US","fr, en, es, it, de",2015.0,10.0,26.0,8.806746e+08
91529,250000000,"Action, Crime, Drama, Thriller","dc comics, crime fighter, terrorist, secret id...",The Dark Knight Rises,112.312950,"Legendary Pictures, Warner Bros., DC Entertain...",2012-07-16,1.084939e+09,165.0,7.6,9106,US,en,2012.0,7.0,16.0,1.084939e+09
93363,260000000,"Action, Adventure, Science Fiction","based on novel, mars, medallion, space travel,...",John Carter,43.926995,Walt Disney Pictures,2012-03-07,2.841391e+08,132.0,6.1,2124,US,en,2012.0,3.0,7.0,2.841391e+08


In [9]:
clean_movies.to_csv('clean_movies_id_appended.csv')

Importing clean movie data

In [11]:
movies = pd.read_csv('clean_movies_id_appended.csv')

Dropping unecessary columns and filling null values

In [13]:
movies = movies.drop(columns = ['revenue', 'release_date', 'release_day'])

movies[['revenue_imputed', 'vote_average']] = movies[['revenue_imputed', 'vote_average']].fillna(0)
movies[['genres', 'keywords', 'production_companies', 'production_ISO', 'spoken_ISO']] = \
    movies[['genres', 'keywords', 'production_companies', 'production_ISO', 'spoken_ISO']].fillna('Unknown')


Creating lists from strings

In [15]:
def parse_list(column):
    return column.split(', ') if isinstance(column, str) else []

movies['genres'] = movies['genres'].apply(parse_list)
movies['keywords'] = movies['keywords'].apply(parse_list)
movies['production_companies'] = movies['production_companies'].apply(parse_list)
movies['production_ISO'] = movies['production_ISO'].apply(parse_list)
movies['spoken_ISO'] = movies['spoken_ISO'].apply(parse_list)

Converting release year into numerical feature and grouping release month into seasons

In [17]:
movies['years_since_1924'] = movies['release_year'] - movies['release_year'].min()

movies.drop(columns = ['release_year'], inplace= True)

movies['winter_release'] = movies['release_month'].isin([12, 1, 2]).astype(int)
movies['spring_release'] = movies['release_month'].isin([3,4,5]).astype(int)
movies['summer_release'] = movies['release_month'].isin([6,7,8]).astype(int)
movies['fall_release'] = movies['release_month'].isin([9,10,11]).astype(int)

movies.drop(columns= ['release_month'], inplace= True)

Standardizing all numeric features

In [19]:
numeric_cols = ['budget', 'popularity', 'runtime', 'vote_average', 'vote_count', 'revenue_imputed', 'years_since_1924']

scaler = StandardScaler()
movies[numeric_cols] = scaler.fit_transform(movies[numeric_cols])

Converting genre into dummy variables

In [21]:
mlb = MultiLabelBinarizer()
genre_df = pd.DataFrame(mlb.fit_transform(movies['genres']), columns = mlb.classes_)

movies = pd.concat([movies,genre_df], axis = 1)

movies.drop(columns = ['genres'], inplace= True)
movies.drop(columns = 'Unknown', inplace= True) #This was created when filling null values

Creating binary features out of spoken ISO and production ISO to represent english language and us production

In [23]:
movies['is_english'] = movies['spoken_ISO'].apply(lambda x: 1 if 'en' in x else 0)
movies['is_us_production'] = movies['production_ISO'].apply(lambda x: 1 if 'US' in x else 0)

movies.drop(columns=['spoken_ISO', 'production_ISO'], inplace=True)

Creating a binary feature if a movie was produced by a major studio

In [25]:
major_studios = {'Walt Disney Pictures', 'Warner Bros.', 'Paramount Pictures', 'Universal Pictures', 'Marvel Studios',
                  'Twentieth Century Fox Film Corporation', 'Touchstone Pictures', 'New Line Cinema', 'Metro-Goldwyn-Mayer (MGM)', 'Columbia Pictures'}

movies['is_major_studio'] = movies['production_companies'].apply(lambda x: 1 if any(studio in major_studios for studio in x) else 0)

movies.drop(columns=['production_companies'], inplace=True)

Storing movie titles and ids in a separate dataframe for later

In [27]:
movies_titles = movies[['movieId', 'original_title']]
movies_titles = movies_titles.set_index('movieId')

Dropping keywords column (Can't get good results with them included) also dropping movie title

In [29]:
movies = movies.drop(columns= ['keywords','original_title'])
movies = movies.set_index('movieId')

Results

In [31]:
movies.head()

,budget,popularity,runtime,vote_average,vote_count,revenue_imputed,years_since_1924,winter_release,spring_release,summer_release,...,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western,is_english,is_us_production,is_major_studio
movieId,,,,,,,,,,,,,,,,,,,,,
72998,4.896324,3.860689,2.527255,1.048134,8.682977,16.440738,0.548730,1,0,0,...,0,0,1,0,0,0,0,1,1,1
53125,6.396770,3.517034,2.853264,0.747886,2.952553,5.274557,0.389531,0,1,0,...,0,0,0,0,0,0,0,1,1,1
136020,5.086857,2.557466,1.875237,0.147391,2.925863,4.783618,1.026327,0,0,0,...,0,0,0,0,0,0,0,1,1,1
91529,5.205940,2.706858,2.666973,1.448464,6.568215,6.032057,0.787529,0,0,1,...,0,0,0,0,1,0,0,1,1,1
93363,5.444106,0.637175,1.130074,-0.052774,1.087417,1.137668,0.787529,0,1,0,...,0,0,1,0,0,0,0,1,1,1


In [32]:
movies.to_csv('movies_preprocessed.csv')

Prepping the User Reviews

Filtering user ratings to only include movies in our data

In [35]:
ratings = pd.read_csv('ratings.csv')

valid_movie_ids = set(clean_movies.index)

filtered_ratings = ratings[ratings['movieId'].isin(valid_movie_ids)]

filtered_ratings = filtered_ratings.drop(columns='timestamp')

filtered_ratings.shape

(15877021, 3)

Only including users with over 200 reviews

In [37]:
user_review_counts = filtered_ratings['userId'].value_counts()
movie_review_counts = filtered_ratings['movieId'].value_counts()

min_reviews = 200

active_users = user_review_counts[user_review_counts >= min_reviews].index

filtered_ratings = filtered_ratings[filtered_ratings['userId'].isin(active_users)]

Number of unique users

In [39]:
filtered_ratings['userId'].nunique()

19999

In [40]:
filtered_ratings.to_csv('user_reviews_200moviesplus.csv', index = False)